# Importação de bibliotecas

* Requests
    * Realizar solicitações para o servidor HTTPS
* Pandas
    * Carregar os arquivos
    * Manipular os DataFrames
    * Armazenar arquivos
* JSON
    * Manipular arquivos do formato JSON

In [1]:
import requests
import pandas as pd
import json

------

# Requests

In [408]:
response_event = requests.get('http://museus.cultura.gov.br/api/event/find?@select=id,name,shortDescription,classificacaoEtaria,occurrences.rule')
response_space = requests.get('http://museus.cultura.gov.br/api/space/find/?@select=id,location,name')

# Ajustando estrutura

## Event

In [ ]:
df_event  = pd.json_normalize(response_event.json())
occurrences = df_event[['occurrences']]
df_event = df_event[['id', 'name', 'shortDescription', 'classificacaoEtaria']]

for j in occurrences.index:
    try:
        occurrences['occurrences'][j] = occurrences['occurrences'][j][0]
    except(IndexError):
        occurrences['occurrences'][j] = {}

occurrences = pd.json_normalize(occurrences['occurrences'])

df_event.rename(columns={'id':'id_event'}, inplace=True)
df_event = pd.concat([df_event, occurrences], axis = 'columns')
df_event.drop(columns=['rule.description', 'rule.day.2', 'rule.day.3','rule.day.4', 'rule.day.0', 'rule.day.5', 'rule.day.6', 'rule.day','rule.day.1', 'id'], inplace=True)

## Space

In [459]:
df_space = pd.json_normalize(response_space.json())
df_space.drop(columns = ['Formulário de Visitação Anual - 2015',
       'Formulário de Visitação Anual - 2016',
       'Formulário de Visitação Anual - 2017',
       'Formulário de Visitação Anual - 2018', 
       'Registro de Museus', 'Formulário de Visitação Anual - 2014',
       'Museu Cadastrado'], inplace=True, errors='ignore')


# Salvando arquivo em JSON

In [461]:
df_event.reset_index().to_json(r'C:\Users\******\files_json\event.json', orient='records')
df_space.reset_index().to_json(r'C:\Users\******\files_json\space.json', orient='records')